In [1]:
from ase.io import read, write
import numpy as np

In [2]:
train_structures = read("./train_tensor/CSD-3k+S546_shift_tensors.xyz",format="extxyz",index=":")

In [3]:
train_structures = train_structures[0:3]
for atoms in train_structures:
    atoms.wrap() 

In [4]:
#check if symmetric

def tensvec_to_irrep(tensor,order=2):
    #flatten tensor?
    fivevec = np.zeros((6))
    fivevec[0] = np.trace(tensor)/np.sqrt(3)
    fivevec[1:4] = tensor[0,1],tensor[1,2],tensor[0,2]
    fivevec[4] = 1/(2*np.sqrt(3)) * (2*tensor[2,2]-tensor[0,0]-tensor[1,1])
    fivevec[5] = 0.5 * (tensor[0,0]-tensor[1,1])
    fivevec[1:] = np.sqrt(2) * fivevec[1:]
    return fivevec
    
def irrep_to_tensvec(b):
    #vector is axx, ayy, azz, axy, ayz, axz
    A = np.array([[1,1,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1],[-1,-1,2,0,0,0],[1,-1,0,0,0,0]])
    print(A)
    b[0] = b[0] * np.sqrt(3)
    b[1:] = b[1:] * 1/np.sqrt(2)
    b[4] = b[4] * 2*np.sqrt(3)
    b[5] = b[5] * 2
    tensvec = np.linalg.solve(A, b)
    return tensvec



In [5]:
def error_metric(irrep_pred,irrep_true):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-5-71c40311fd9e>, line 2)

In [ ]:
tens = train_structures.arrays["cs_tensor"][0]

In [ ]:
tens = tens.reshape((3,3))

In [ ]:
tens

In [ ]:
irrep_vec = tensvec_to_irrep(tens)

In [ ]:
irrep_vec[:,np.newaxis].shape

In [ ]:
irrep_to_tensvec(irrep_vec[:,np.newaxis])

In [ ]:
np.ndarray(5)

In [ ]:
this = np.zeros(5)

In [ ]:
this[0:3] = 1,2,3

In [ ]:
this

In [6]:
from rascal.representations import SphericalCovariants

In [7]:
hypers = {"soap_type": "LambdaSpectrum",
          "interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 6,
          "max_angular": 4,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"ShiftedCosine",
          "cutoff_smooth_width": 0.5,
          "normalize": False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4),
          #"optimization": dict(Spline=dict(accuracy=1.0e-05))
          #"global_species":[1,6,7,8]
          "covariant_lambda":2,
          "inversion_symmetry":False
         }

In [8]:
calculator = SphericalCovariants(**hypers)
atoms_list_train = calculator.transform(train_structures)

In [9]:
X_train = calculator.transform(structures_train).get_features(calculator)

NameError: name 'structures_train' is not defined

In [ ]:
X_train.shape

In [10]:
# retrieve (N_environ, 9) matrix
# build lambda 0 and lambda 2 features
# reshape lambda 2 features into (N_environ, 5, N_feature_vectors)
# build K_NN like K_MM matrix
# do prediction

In [11]:
from helpers import *

In [105]:
PATH_TRAIN = "./train_tensor/CSD-3k+S546_shift_tensors.xyz"
PATH_TEST = "./test_tensor/CSD-500+104-7_shift_tensors.xyz"

structures_train, tensors_train = load_CSD_data(PATH_TRAIN,prop_string="cs_tensor",random_subsample=10)
structures_test, tensors_test = load_CSD_data(PATH_TRAIN,prop_string="cs_tensor",random_subsample=100)

In [110]:
tensors_vecs = np.vstack([tensvec_to_irrep(tens)  for tens in tensors_train.reshape(-1,3,3)])

In [17]:
#tensors_vecs[:,1:] are lambda=2 components
#tensor_vecs[:,0] are lambda=0 components

In [18]:
from rascal.representations import SphericalExpansion
from rascal.utils import ClebschGordanReal, compute_lambda_soap, spherical_expansion_reshape

In [122]:
# to obtain lambda=2 spherical covariant features:

# calculate spherical expansion of neighbour density
hypers = {"interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 9,
          "max_angular": 9,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"ShiftedCosine",
          "cutoff_smooth_width": 0.5,
          "compute_gradients":False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4),
          #"optimization": dict(Spline=dict(accuracy=1.0e-05))
          #"expansion_by_species_method":'structure wise'
         }



spex = SphericalExpansion(**hypers)

#why is coefficient scaling nescessary? 
feat_scaling = 1e6            # just a scaling to make coefficients O(1)
feats = spex.transform(structures_train).get_features(spex)
ref_feats = feat_scaling*spherical_expansion_reshape(feats, **hypers)
feats_test = spex.transform(structures_test).get_features(spex)
ref_feats_test = feat_scaling*spherical_expansion_reshape(feats_test, **hypers)

In [113]:
CG = ClebschGordanReal(lmax=hypers["max_angular"])

In [114]:
sel_lambda = 2
lsoap_utils = compute_lambda_soap(ref_feats, CG, sel_lambda, 0)

In [115]:
X_element_wise, y_element_wise = return_element_wise_environments(lsoap_utils,spex,structures_train,y=tensors_vecs,select=1)

In [116]:
X_element_wise_reshaped = X_element_wise.reshape(X_element_wise.shape[0],5,-1)

In [118]:
#from https://github.com/gabor1/chemrev-gpr/blob/main/water/water.ipynb

def f2v(vec):
    t = ()
    for s in vec.shape:
        t += (s//5,5)
    return vec.reshape(t)

def v2f(vec):
    t = ()
    for s in vec.shape[:-1:2]:        
        t += tuple([s*5])
    if len(vec.shape)%2==1: #if the number of dimensions is uneven
        t += tuple([-1])
    #print(t)
    return vec.reshape(t)

In [119]:
lKMM = f2v(v2f(X_element_wise_reshaped) @ v2f(X_element_wise_reshaped).T)

In [120]:
wlsparse = f2v( np.linalg.lstsq(v2f(lKMM) + 1e-8*  v2f(lKMM), v2f(y_element_wise[:,1:]), rcond=None)[0] )

In [121]:
lKM = f2v(v2f() @ v2f(X_element_wise_reshaped).T)

(398, 5)

In [99]:
ylpred_sparse = f2v(v2f(lKMM) @ v2f(wlsparse) )

In [68]:
ylpred_sparse.shape

(336, 5)

In [254]:
tensors_vecs[:,1:].shape

(1024, 5)

In [100]:
from sklearn.metrics import mean_squared_error

In [123]:
mean_squared_error(np.linalg.norm(y_element_wise[:,1:],axis=1), np.linalg.norm(ylpred_sparse, axis=1))

ValueError: Found input variables with inconsistent numbers of samples: [398, 336]

In [102]:
y_element_wise[:,1:].shape

(336, 5)

In [103]:
ylpred_sparse.shape

(336, 5)

In [208]:
v2f(lKMM).shape

(4640, 4640)

In [207]:
tensors_vecs[:,1:].shape

(18969, 5)

In [ ]:
#wg = np.linalg.lstsq( kgauss[itrain][:,itrain]    + 1e-4*np.eye(len(itrain)), y[itrain], rcond=None)[0]
#yp_g = (kgauss[:,itrain]) @ wg